In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
file_path = 'CR1000Y/CR1000Y-12.csv'

Sample_Spectra = pd.read_csv(file_path)
Sample_Spectra = Sample_Spectra.rename({' intensity':'Intensities'},axis=1)


Sample_Spectra_Plot = figure(title = 'Raw Sample Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Sample_Spectra_Plot.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
Sample_Spectra_Plot.width = 1200
Sample_Spectra_Plot.height = 600

# print(Sample_Spectra)
# show (Sample_Spectra_Plot)

In [4]:
#https://pybaselines.readthedocs.io/en/latest/api/pybaselines/index.html

# base,_ = pl.drpls(Sample_Spectra.Intensities)

# Sample_Spectra_Baseline = figure(title = 'Corrected Baseline in hLIBS Spectra', x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
# Sample_Spectra_Baseline.line(Sample_Spectra.wavelength,Sample_Spectra.Intensities , line_width = 2, color ="red" )
# # Sample_Spectra_Baseline.line(Sample_Spectra.wavelength,base , line_width = 2, color ="red" )
# Sample_Spectra_Baseline.line(Sample_Spectra.wavelength,(Sample_Spectra.Intensities - base) , line_width = 2, color ="green" )

# Corrected_Baseline_Spectra = pd.DataFrame()
# Corrected_Baseline_Spectra['wavelength']= Sample_Spectra.wavelength
# Corrected_Baseline_Spectra['Intensities'] = (Sample_Spectra.Intensities-base)
# print(Corrected_Baseline_Spectra)

# Sample_Spectra_Baseline.width = 1200
# Sample_Spectra_Baseline.height = 600
# show(Sample_Spectra_Baseline)
# print(Sample_Spectra_Baseline)



In [5]:
# Normalization_Intensity = max(Sample_Spectra.Intensities)
# print(Normalization_Intensity)

# ###########################################################################

# matching_row = Sample_Spectra[Sample_Spectra['Intensities'] == Normalization_Intensity]
# print(matching_row)

###########################################################################
#For CR300LA - Normalizing the spectra by Matrix - Fe peak 259.90 - 259.8999999999988 , 238.16666666666336 , 263.10000000000133

target_wavelength = 263.10000000000133  #this value need to match exactly with CSV
Normalization_Intensity = Sample_Spectra.loc[Sample_Spectra['wavelength'] == target_wavelength, 'Intensities'].iloc[0]
print (Normalization_Intensity)


25840.454505297545


In [6]:
Normalized_Spectra = pd.DataFrame()
Normalized_Spectra['wavelength']= Sample_Spectra.wavelength
Normalized_Spectra['Intensities'] = (Sample_Spectra.Intensities / Normalization_Intensity)
print(Normalized_Spectra)

Nor = Normalized_Spectra.loc[Normalized_Spectra['wavelength'] == target_wavelength, 'Intensities'].iloc[0]
print(Nor)


       wavelength  Intensities
0      180.000000          0.0
1      180.033333          0.0
2      180.066667          0.0
3      180.100000          0.0
4      180.133333          0.0
...           ...          ...
23426  960.866667          0.0
23427  960.900000          0.0
23428  960.933333          0.0
23429  960.966667          0.0
23430  961.000000          0.0

[23431 rows x 2 columns]
1.0


In [7]:
Selected_data = Normalized_Spectra[(Normalized_Spectra['wavelength'] >= 202.4) & (Normalized_Spectra['wavelength'] <= 202.7)]
# print(Selected_data)
height = None
Peak_Prominenace = None #(RMS_intensity_noise + Threshold)  #Uses S/N ratio , as comapred to other peaks
minimum_distance = None   #minimum distance between two peaks

Selected_data_peak , _ = fp(Selected_data.Intensities,height=height,prominence=Peak_Prominenace,distance=minimum_distance)
Selected_Sample_Spectra = Selected_data.iloc[Selected_data_peak] 
print(Selected_Sample_Spectra)


Selected_data_Plot = figure(title = 'Selected Data Spectra' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')
Selected_data_Plot.line(Selected_data.wavelength,Selected_data.Intensities , line_width = 2, color ="red" )
Selected_data_Plot.circle(Selected_Sample_Spectra.wavelength, Selected_Sample_Spectra.Intensities , color = 'green' , size = 8)

Selected_data_Plot.width = 250

Selected_data_Plot.height = 400
show (Selected_data_Plot)

     wavelength  Intensities
677  202.566667     0.005088
